<h1>Desafio análise de dados - Wellbe</h1>
Desenvolvido por <b>Guilherme Antunes</b>

In [343]:
#importar bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from datetime import datetime
import sqlalchemy

In [344]:
#carregar csv / txt
df = pd.read_csv(r"D:\Guilherme\Desafio2.txt", sep=';', encoding='latin1')
df.head()

,Código,Custo do afastamento,Identificação,Funcionário,Cargo,Data do Atestado,Especialidade,Motivo
0,1036743,"20,4",NaN,Anonimo 1,ASSISTENTE DE IMPLANTACAO,29/05/2019,Exames,Exames
1,1036742,23,NaN,Anonimo 1,ASSISTENTE DE IMPLANTACAO,23/05/2019,Neurologia pediátrica,Acompanhamento familiar
2,1036741,"22,8",NaN,Anonimo 1,ASSISTENTE DE IMPLANTACAO,17/05/2019,NaN,Consulta médica
3,1036740,"33,9",NaN,Anonimo 1,ASSISTENTE DE IMPLANTACAO,16/05/2019,Exames,Exames
4,1036739,22,NaN,Anonimo 1,ASSISTENTE DE IMPLANTACAO,06/05/2019,NaN,Consulta médica


In [345]:
#Tipos de variáveis
df.dtypes

Código                    int64
Custo do afastamento     object
Identificação           float64
Funcionário              object
Cargo                    object
Data do Atestado         object
Especialidade            object
Motivo                   object
dtype: object

In [346]:
#Quantidade de linhas e colunas
df.shape

(91, 8)

In [347]:
#Substitui campo vazio em Data do Atestado
df["Data do Atestado"].fillna('-', inplace=True)

In [348]:
#Substitui traço por uma data aleatória
df['Data do Atestado'] = df['Data do Atestado'].apply(lambda x: str(x).replace("-",'29/05/2019'))

In [349]:
#Converter data do atestado para o datetime usando to_datetime
df['Data do Atestado'] = pd.to_datetime(df['Data do Atestado'], format="%d/%m/%Y")

In [350]:
#Substituindo a vírgula para ponto, para poder converter para float
df['Custo do afastamento'] = df['Custo do afastamento'].apply(lambda x: str(x).replace(",","."))

In [351]:
#Atribuindo tipo float para a coluna Custo do afastamento
df['Custo do afastamento'] = df['Custo do afastamento'].astype('float64')

In [352]:
#Média com todos os valores do custo para verificar se deu certo a conversão
df['Custo do afastamento'].mean()

6511.606111111111

In [353]:
#Deletar a coluna identificação que não possui nenhuma informação
del df['Identificação']

In [354]:
#Substitui campo vazio por outros em Cargo
df["Cargo"].fillna('Outros', inplace=True)

In [355]:
#Substitui campo vazio por outras em Especialidade
df["Especialidade"].fillna('Outras', inplace=True)

In [356]:
#Substitui campo vazio por outros em Motivos
df["Motivo"].fillna('Outros', inplace=True)

In [357]:
#Substitui campo vazio por 310 em custo do afastamento, valor em comum entre Especialidade e Motivo
df["Custo do afastamento"].fillna(310, inplace=True)

In [358]:
#Inserindo a coluna ID no início 
df.insert(0, 'ID', value=range(len(df)), allow_duplicates=False)

In [359]:
df.dtypes

ID                               int32
Código                           int64
Custo do afastamento           float64
Funcionário                     object
Cargo                           object
Data do Atestado        datetime64[ns]
Especialidade                   object
Motivo                          object
dtype: object

In [360]:
df.head()

,ID,Código,Custo do afastamento,Funcionário,Cargo,Data do Atestado,Especialidade,Motivo
0,0,1036743,20.4,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-29,Exames,Exames
1,1,1036742,23.0,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-23,Neurologia pediátrica,Acompanhamento familiar
2,2,1036741,22.8,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-17,Outras,Consulta médica
3,3,1036740,33.9,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-16,Exames,Exames
4,4,1036739,22.0,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-06,Outras,Consulta médica


In [413]:
#Alterando os nomes das colunas
df.columns = ['id', 'codigo', 'custo', 'funcionario', 'cargo', 'data_do_afastamento', 'especialidade', 'motivo']

In [362]:
#conexão com o banco de dados
conectar = sqlalchemy.create_engine('mysql+pymysql://root@localhost:3306/desafio_wellbe')

In [363]:
#Enviando dataframe para o banco
df.to_sql(
    name = 'afastamentos',
    con = conectar,
    index = False,
    if_exists ='append'
)

In [364]:
#Lendo a tabela afastamentos do banco
df = pd.read_sql_table('afastamentos', conectar)

In [365]:
df.head()

,id,codigo,custo,funcionario,cargo,data_do_afastamento,especialidade,motivo
0,0,1036743,20.4,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-29,Exames,Exames
1,1,1036742,23.0,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-23,Neurologia pediátrica,Acompanhamento familiar
2,2,1036741,22.8,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-17,Outras,Consulta médica
3,3,1036740,33.9,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-16,Exames,Exames
4,4,1036739,22.0,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-06,Outras,Consulta médica



<b>1) Qual departamento gastou mais em afastamentos?<b>

In [375]:
#Cria a consulta SQL selecionando o cargo, somando o custo correspondente e ordenando de maneira decrescente
consulta1 = '''SELECT cargo AS Cargo, SUM(custo) AS Custo 
FROM afastamentos GROUP BY cargo ORDER BY custo desc'''

In [376]:
#Lê a consulta criada guardando em uma variável
resultado1 = pd.read_sql_query(consulta1, conectar)

In [377]:
#mostra o departamento que mais gastou em ordem decrescente 
display(resultado1)

,Cargo,Custo
0,ANALISTA CONTABIL II,101365.00
1,ASSISTENTE DE IMPLANTACAO,67995.10
2,TECNICO SEGUROS VG I,65527.00
3,ANALISTA ESTUDOS E COTACAO II,58068.00
4,ASSISTENTE CONTROLE OPERACIONAL,57770.00
5,ANALISTA ESTUDOS E COTACAO I,57264.00
6,Outros,28950.00
7,TECNICO SEGUROS IV,28715.00
8,TECNICO DE SEGUROS I,19932.00
9,ANALISTA SINISTROS I,19048.00


<b>2) Qual especialidade teve o maior gasto?<b>

In [437]:
#Cria a consulta SQL selecionando especialide, somando o custo correspondente e ordenando de maneira decrescente
consulta2 = '''SELECT especialidade AS Especialidade, SUM(custo) AS Custo 
FROM afastamentos GROUP BY Especialidade ORDER BY custo desc'''

In [435]:
#Lê a consulta criada guardando em uma variável
resultado2 = pd.read_sql_query(consulta2, conectar)

In [409]:
#mostra a especialidade que mais gastou em ordem decrescente 
display(resultado2)

,Especialidade,Custo
0,Outras,124317.25
1,Odontologia,116072.00
2,Ortopedia,39479.00
3,Exames,38046.30
4,Ginecologia/Obstetricia,37466.00
5,Pediatria,28760.00
6,Otorrinolaringologia,28730.00
7,Gastroenterologia,28092.00
8,Obstetricia,27645.00
9,Radiologia,19356.00


<b>3) Qual foi o principal motivo de afastamento?<b>

In [378]:
#Cria a consulta SQL selecionando e contando o motivo e ordenando de maneira decrescente
consulta3 = '''SELECT motivo AS Motivo, count(motivo) AS Quantidade 
FROM afastamentos GROUP BY Motivo ORDER BY Quantidade desc''' 

In [379]:
#Lê a consulta criada guardando em uma variável
resultado3 = pd.read_sql_query(consulta3, conectar)

In [380]:
#mostra o principal motivo de afastamento em ordem decrescente 
display(resultado3)

,Motivo,Quantidade
0,Consulta médica,29
1,Exames,21
2,Consulta odontológica,10
3,Acompanhamento familiar,7
4,Dor/Doença,7
5,Outros,6
6,Cirurgia Odontológica,2
7,Acidente,2
8,Tratamento,2
9,Licença gala,1


In [415]:
#Salva em csv no disco planilha atualizada
df.to_csv('Desafio_modificado.csv', encoding = 'utf-8', index = False)

In [417]:
dataset = pd.read_csv('Desafio_modificado.csv')
dataset.head()

,id,codigo,custo,funcionario,cargo,data_do_afastamento,especialidade,motivo
0,0,1036743,20.4,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-29,Exames,Exames
1,1,1036742,23.0,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-23,Neurologia pediátrica,Acompanhamento familiar
2,2,1036741,22.8,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-17,Outras,Consulta médica
3,3,1036740,33.9,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-16,Exames,Exames
4,4,1036739,22.0,Anonimo 1,ASSISTENTE DE IMPLANTACAO,2019-05-06,Outras,Consulta médica
